# Former Teams of Players in the Playoffs

What playoff team should fans of non-playoff teams root for, based on which playoff team has the most former members of your favorite team? 

Data from: https://www.fangraphs.com/leaders/splits-leaderboards?splitArr=&strgroup=career&statgroup=1&startDate=2014-01-01&endDate=2019-09-27&filter=&position=B&statType=player&autoPt=true&sort=22,1&pg=0&players=&splitArrPitch=&splitTeams=false

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.width', 1000)

The 10 playoff teams: 

In [2]:
playoff_teams = [
    'NYY', 'TBR', 'MIN', 'HOU', 'OAK'
    , 'ATL', 'WSN', 'STL', 'MIL', 'LAD'
]

teams = [
    'ARI', 'ATL', 'BAL', 'BOS', 'CHC', 'CHW', 'CIN', 'CLE', 'COL', 'DET', 'HOU', 'KCR', 'LAA', 'LAD'
    , 'MIA', 'MIL', 'MIN', 'NYM', 'NYY', 'OAK', 'PHI', 'PIT', 'SDP', 'SEA', 'SFG', 'STL', 'TBR', 'TEX'
    , 'TOR', 'WSN'
]

not_playoff_teams = [t for t in teams if t not in playoff_teams]

## Data - Past Seasons

Player seasons from 2001 through 2019, split by team (CC Sabathia made his debut in 2001, the earliest debut of any player on a playoff team in 2019). Players must have at least 10 plate appearances or batters faced to count. 

### Batting

https://www.fangraphs.com/leaders/splits-leaderboards?splitArr=&strgroup=career&statgroup=1&startDate=2001-01-01&endDate=2019-09-30&filter=PA%7Cgt%7C10&position=B&statType=player&autoPt=false&sort=22,1&pg=0&players=&splitArrPitch=&splitTeams=true

In [3]:
batting = pd.read_csv('fangraphs_batting.csv')

### Pitching

https://www.fangraphs.com/leaders/splits-leaderboards?splitArr=&strgroup=career&statgroup=1&startDate=2001-01-01&endDate=2011-01-01&filter=TBF%7Cgt%7C10&position=P&statType=player&autoPt=false&sort=22,1&pg=0&players=&splitArrPitch=&splitTeams=true

https://www.fangraphs.com/leaders/splits-leaderboards?splitArr=&strgroup=career&statgroup=1&startDate=2011-01-01&endDate=2019-09-29&filter=TBF%7Cgt%7C10&position=P&statType=player&autoPt=false&sort=22,1&pg=0&players=&splitArrPitch=&splitTeams=true

In [4]:
pitching_2001 = pd.read_csv('fangraphs_pitchers_2001_to_2010.csv')
pitching_2011 = pd.read_csv('fangraphs_pitchers_2011_to_2019.csv')
pitching = pd.concat([pitching_2001, pitching_2011])

We need one metric for both pitchers and batters. Fangraphs doens't provide "games" for pitchers in this dataset. Let's use Plate Appearances by batters and Batters Faced by pitchers, and combine them into one metric, `appearances`. 

In [5]:
batting['appearances'] = batting['PA']
pitching['appearances'] = pitching['TBF']

Combine batting and pitching data into one de-duplicated dataframe, with one row per player-season-team. 

In [6]:
columns_to_union = ['playerId', 'Season', 'Name', 'Tm', 'appearances']
players_with_duplicates = pd.concat([pitching[columns_to_union], batting[columns_to_union]])
players = players_with_duplicates.groupby(['playerId', 'Season', 'Name', 'Tm']).appearances.sum().reset_index()

## Data - Current Playoff Players

A "playoff player" is defined as a player who has made an appearance in September 2019 for one of the 10 teams in the playoffs. 

### Batting - September

https://www.fangraphs.com/leaders/splits-leaderboards?splitArr=&strgroup=career&statgroup=1&startDate=2019-09-01&endDate=2019-09-30&filter=&position=B&statType=player&autoPt=false&sort=22,1&pg=0&players=&splitArrPitch=&splitTeams=true

In [7]:
batting_sept = pd.read_csv('fangraphs_batting_sept_2019.csv')

### Pitching - September

https://www.fangraphs.com/leaders/splits-leaderboards?splitArr=&strgroup=career&statgroup=1&startDate=2019-09-01&endDate=2019-09-30&filter=&position=P&statType=player&autoPt=false&sort=22,1&pg=0&players=&splitArrPitch=&splitTeams=true

In [8]:
pitching_sept = pd.read_csv('fangraphs_pitching_sept_2019.csv')

Combine September batting and pitching data into one de-duplicated dataframe, with one row per player-game.

In [9]:
cols_to_union_sept = ['playerId', 'Name', 'Date', 'Tm']
sept_games = pd.concat([batting_sept[cols_to_union_sept], pitching_sept[cols_to_union_sept]]).drop_duplicates()

Create a dataframe with every playoff player and their team. Use the most recent appearance in September to connect player and team. 

In [10]:
most_recent = pd.DataFrame(sept_games.groupby('playerId').Date.max().reset_index())
most_recent_team = pd.merge(most_recent, sept_games, how='inner', on=['playerId', 'Date'])

playoff_players = most_recent_team[most_recent_team.Tm.isin(playoff_teams)][['playerId', 'Name', 'Tm']].reset_index()

The number of appearances that every player has made for each team. One row per player-team. 

In [11]:
# player_team_appearances = players.groupby(['playerId', 'Tm']).appearances.sum().reset_index()
players['first_season_with'] = players['Season']
players['last_season_with'] = players['Season']

player_team_appearances = players.groupby(['playerId', 'Tm']).agg({
    'appearances': sum
    , 'first_season_with': min
    , 'last_season_with': max
}).reset_index()

For every player, a list of the teams they've ever played for. Narrow this dataframe to only players on playoff teams. 

In [12]:
player_teams = players.groupby('playerId')['Tm'].apply(list).reset_index(name='team_list')
playoff_player_teams = pd.merge(playoff_players, player_teams, how='inner', on='playerId')

In [13]:
def former_players_from_team(team, playoff_player_teams, player_team_appearances):
    """
    For one non-playoff team, return the teams that its players have formerly played for
    """
    playoff_from_team_list = []
    for idx, row in playoff_player_teams.iterrows(): 
        if team in row.team_list: 
            playoff_from_team_list += [{
                'playerId': row.playerId
                , 'Name': row.Name
                , 'playoff_team': row.Tm
            }]
            
    playoff_from_team = pd.DataFrame(playoff_from_team_list)
    playoff_from_team_app = player_team_appearances[
        (player_team_appearances['playerId'].isin(playoff_from_team.playerId))
        & (player_team_appearances['Tm'] == team)
    ]
    
    return pd.merge(playoff_from_team_app, playoff_from_team, how='inner', on='playerId')

## Iterate through non-playoff teams

Write output to one text file per non-playoff team. 

In [14]:
best_team = {}

for t in not_playoff_teams:

    df = former_players_from_team(t, playoff_player_teams, player_team_appearances)

    df_app = pd.DataFrame(
        df.groupby('playoff_team').appearances.sum().reset_index(name='player_appearances')
        ).sort_values(by='player_appearances', ascending=False)

    best_team[t] = df_app.loc[df_app['player_appearances'].idxmax()]['playoff_team']

    df.rename(columns={
        'first_season_with': f'first_season_with_{t}'
        , 'last_season_with': f'last_season_with_{t}'
    }, inplace=True)
    
    with open(f'non_playoff_teams/{t}.txt','w+') as f:
    
        f.write('Team not in playoffs: {}\n'.format(t))

        f.write(f'\n\nPlayoff teams with former {t} players, by number of {t} appearances:\n\n')
        f.write(df_app.to_string(index=False))
        f.write('\n')

        columns_to_write = ['Tm', 'Name', 'appearances', f'first_season_with_{t}', f'last_season_with_{t}']

        for idx, row in df_app.iterrows(): 
            f.write('\n\nAppearances that current {} players made for {}: {}\n'.format(
                row.playoff_team, t, row.player_appearances))
            f.write(df[df.playoff_team == row.playoff_team][columns_to_write]
                  .sort_values(by='appearances', ascending=False).to_string(index=False))

In [15]:
best_team

{'ARI': 'WSN',
 'BAL': 'ATL',
 'BOS': 'MIL',
 'CHC': 'LAD',
 'CHW': 'TBR',
 'CIN': 'OAK',
 'CLE': 'NYY',
 'COL': 'NYY',
 'DET': 'HOU',
 'KCR': 'MIL',
 'LAA': 'WSN',
 'MIA': 'NYY',
 'NYM': 'WSN',
 'PHI': 'NYY',
 'PIT': 'TBR',
 'SDP': 'MIL',
 'SEA': 'NYY',
 'SFG': 'MIN',
 'TEX': 'MIN',
 'TOR': 'NYY'}